In [11]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
# Import API key
from api_keys import g_key

In [12]:
#Store Part I results into DataFrame
#Load the csv exported in Part I to a DataFrame
weather_data = pd.read_csv("output_data/weather_data.csv", encoding="utf-8")
weather_data.head()

,Unnamed: 0,City,Country,Date,Latitude,Longitude,Cloudiness (%),Humidity (%),Temperature (F),Wind Speed (mph)
0,0,Ribeira Grande,PT,1578326932,38.52,-28.70,40,93,64.4,13.87
1,1,Punta Arenas,CL,1578326803,-53.16,-70.91,75,71,57.0,20.80
2,2,Puerto Ayora,EC,1578326938,-0.74,-90.35,40,65,82.4,12.75
3,3,Kapaa,US,1578326939,22.08,-159.32,75,83,75.2,19.46
4,4,Skibbereen,IE,1578326940,51.55,-9.27,40,81,46.4,11.41


In [13]:
#Humidity Heatmap
#Configure gmaps.
gmaps.configure(api_key=g_key)
#Use the Lat and Lng as locations and Humidity as the weight.
locations = weather_data[["Latitude", "Longitude"]]
#Drop any rows will null values
humidity = weather_data["Humidity (%)"].astype(float)
maxhumidity = humidity.max()

fig = gmaps.figure()

#Add Heatmap layer to map.
heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                    dissipating=False, max_intensity=maxhumidity,
                                    point_radius=2)
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [14]:
#Create new DataFrame fitting weather criteria
#Narrow down the cities to fit weather conditions.

ideal_weather_df = pd.DataFrame(weather_data, columns = ["City", "Country", "Latitude", "Longitude", 
                                                         "Temperature (F)", "Wind Speed (mph)", "Cloudiness (%)"])

temperature = (weather_data["Temperature (F)"] <= 80) & (ideal_weather_df["Temperature (F)"] > 70)
wind_speed = weather_data["Wind Speed (mph)"] < 10
cloudiness = weather_data["Cloudiness (%)"] == 0

ideal_weather_df = ideal_weather_df[temperature & wind_speed & cloudiness]
ideal_weather_df

,City,Country,Latitude,Longitude,Temperature (F),Wind Speed (mph),Cloudiness (%)
91,Marzuq,YE,14.40,46.47,74.34,9.62,0
137,Sur,OM,22.57,59.53,74.50,5.30,0
310,Jawhar,IN,19.91,73.23,70.29,4.05,0
345,Champasak,LA,14.89,105.88,70.90,1.86,0
347,Iquique,CL,-20.22,-70.14,77.00,6.93,0
412,Mapastepec,MX,15.44,-92.90,78.17,5.06,0
424,Muttupet,IN,10.40,79.48,74.91,6.31,0


In [15]:
#Hotel Map
#Store into variable named hotel_df.
hotel_df = []
ideal_weather_df["Hotel Name"] = ""
ideal_weather_df

,City,Country,Latitude,Longitude,Temperature (F),Wind Speed (mph),Cloudiness (%),Hotel Name
91,Marzuq,YE,14.40,46.47,74.34,9.62,0,
137,Sur,OM,22.57,59.53,74.50,5.30,0,
310,Jawhar,IN,19.91,73.23,70.29,4.05,0,
345,Champasak,LA,14.89,105.88,70.90,1.86,0,
347,Iquique,CL,-20.22,-70.14,77.00,6.93,0,
412,Mapastepec,MX,15.44,-92.90,78.17,5.06,0,
424,Muttupet,IN,10.40,79.48,74.91,6.31,0,


In [16]:
#Hit the Google Places API for each city's coordinates
for index, row in ideal_weather_df.iterrows():
    latitude = row["Latitude"]
    longitude = row["Longitude"]
#Set parameters to search for hotels with 5000 meters.
    params = {
        "location": f"{latitude},{longitude}",
        "radius": 5000,
        "types" : "hotel",
        "key": g_key
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    #run a request
    requested = requests.get(base_url, params=params)
    jsn = requested.json()
    try:
        hotel_df.append(jsn['results'][0]['name'])
    except:
        hotel_df.append("")
ideal_weather_df["Hotel Name"] = hotel_df
ideal_weather_df = ideal_weather_df.dropna(how='any')
ideal_weather_df

,City,Country,Latitude,Longitude,Temperature (F),Wind Speed (mph),Cloudiness (%),Hotel Name
91,Marzuq,YE,14.40,46.47,74.34,9.62,0,Al Hujayrah
137,Sur,OM,22.57,59.53,74.50,5.30,0,Sur
310,Jawhar,IN,19.91,73.23,70.29,4.05,0,Jawhar
345,Champasak,LA,14.89,105.88,70.90,1.86,0,Muang Champassak
347,Iquique,CL,-20.22,-70.14,77.00,6.93,0,Iquique
412,Mapastepec,MX,15.44,-92.90,78.17,5.06,0,Mapastepec
424,Muttupet,IN,10.40,79.48,74.91,6.31,0,Muttupet


In [17]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

In [25]:
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_df = [info_box_template.format(**row) for index, row in ideal_weather_df.iterrows()]
marker_locations = ideal_weather_df[["Latitude", "Longitude"]]

In [36]:
#Configure gmaps
gmaps.configure(api_key=g_key)
#Use the Lat and Lng as locations and Humidity as the weight.
locations = weather_data[["Latitude", "Longitude"]]
#Drop any rows will null values
humidity = weather_data["Humidity (%)"].astype(float)
maxhumidity = humidity.max()

fig = gmaps.figure()

#Add Heatmap layer to map.
heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                    dissipating=False, max_intensity=maxhumidity,
                                    point_radius=2)
fig.add_layer(heatmap_layer)

# Add marker layer ontop of heat map
marker_locations = [
    (14.40, 46.47),
    (22.57, 59.53),
    (19.91, 73.23),
    (14.89, 105.88),
    (-20.22, -70.14),
    (15.44, -92.90),
    (10.40, 79.48)
]

#fig = gmaps.figure()

markers = gmaps.marker_layer(marker_locations)
fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))